In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# USER INPUTS

YEARS = range(2020, 2099)
#LONLAT = (36.817223, -1.286389)   # (x, y) for Nairobi
LONLAT = (76.383896, 10.426311,) # (x, y) for Kerala

current_geom = ee.Geometry.Point(LONLAT)

In [ ]:
maxprecip_probs = ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/maxprecip')

In [ ]:
def expected_value(year, geom, probdata):
# Return expected value of hazard for given year
    probs = probdata.filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=geom, scale=250, bestEffort=True).getInfo()
    if np.sum([float(i) for i in probs.values()]) == 0:
        return 0, 1
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0].replace('neg', '-')) + float(i[1].replace('neg', '-'))) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    for idx in range(len(mag_range)):
        mag_pair = mag_range[idx]
        if ev >= float(mag_pair[0].replace('neg', '-')) and ev < float(mag_pair[1].replace('neg', '-')):
            break
    key = list(probs.keys())[idx]
    return ev

In [ ]:
maxprecip_evs = [expected_value(y, current_geom, maxprecip_probs) for y in YEARS]

In [ ]:
df = pd.DataFrame({'year': YEARS, 'max daily precip':maxprecip_evs})

In [ ]:
df

In [ ]:
g = sns.relplot(x="year", y="max daily precip", kind="line", data=df)
g.figure.autofmt_xdate()

In [ ]:
startyear = df[df.year == 2020]['max daily precip'].iloc[0]
endyear = df[df.year == 2050]['max daily precip'].iloc[0]
print(startyear,endyear)

In [ ]:
pctChange = (endyear /startyear) - 1
print(pctChange)